<a href="https://colab.research.google.com/github/nikolmash/compling-it2020/blob/master/hw6_compling%26it.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gensim --upgrade -q
!pip install pymorphy2 -q


import pandas as pd
import numpy as np
import pymorphy2
import re
from string import punctuation
from gensim.models import FastText
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split

%tensorflow_version 1.x

import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Bidirectional, TimeDistributed, Conv1D, Flatten, concatenate
from keras.models import Model
from keras.optimizers import Adam

TensorFlow 1.x selected.


Using TensorFlow backend.


In [2]:
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv

--2021-02-21 14:46:08--  https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/r6u59ljhhjdg6j0/negative.csv [following]
--2021-02-21 14:46:09--  https://www.dropbox.com/s/raw/r6u59ljhhjdg6j0/negative.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uce5dd90142fbff04ab3f13be51c.dl.dropboxusercontent.com/cd/0/inline/BJXsnGYELOWO62vL0JYaZ8OyXfSnnBqn19nSDfNAZbblC4E5oOcfZyiXhW-qSTTdFJNwX0iE9HXr3EwwWIxM6dFmW2_HdntX2Rr-H5BT7nJJXg/file# [following]
--2021-02-21 14:46:09--  https://uce5dd90142fbff04ab3f13be51c.dl.dropboxusercontent.com/cd/0/inline/BJXsnGYELOWO62vL0JYaZ8OyXfSnnBqn19nSDfNAZbblC4E5oOcfZyiXhW-qSTTdFJNwX0iE9HXr3EwwWIxM6dFmW2_HdntX2Rr-H5BT7nJJXg/file
Resolving uce5dd

In [2]:
cols = ['id', 'date', 'name', 'text', 'type', 'rep', 'rtw', 'fav', 'stcount', 'foll', 'frien', 'listcount'] 

df1 = pd.read_csv('positive.csv', delimiter=';', header=None, names=cols)
df2 = pd.read_csv('negative.csv', delimiter=';', header=None, names=cols)

df = pd.concat([df1,df2], ignore_index=True)
df.sample(10)

,id,date,name,text,type,rep,rtw,fav,stcount,foll,frien,listcount
17884,409375369251147776,1386437668,Hermiiona,"Ахаха, ходить в 23:30 в магазин, чтобы заплати...",1,0,0,0,120,56,66,0
76415,410698319250210816,1386753084,pillSa5,"США сознались, что на территории России имеют ...",1,0,0,0,26465,417,320,10
78997,410726836478681089,1386759883,MAMKA_XA33bI,"В новом выпуске ""ВЗ"" с одной стороны 1D, с дру...",1,0,0,0,30670,4433,3486,30
193919,419422148659539968,1388833007,deathbedz,Еще у меня в ленте в инстаграме было две дуры ...,-1,0,0,0,18998,548,54,9
103413,411111565061263360,1386851609,fudoko_senaemi,я два месяца пил витамины и в последний день п...,1,0,0,0,45005,387,384,6
11566,409301512393744384,1386420059,laura16_sl,"А вот и мы) Аделина, Андрео, Самуэль и Валерио...",1,0,0,0,2880,149,97,1
164668,415111875509374976,1387805358,zizizacuquka,"Бесит, когда учителя добавляются в друзья! И о...",-1,0,0,0,45,15,27,0
205947,421837982711046145,1389408987,pirogovaov,до сих пор болит не могу(((((ну всё щитай отра...,-1,0,0,0,4,0,0,0
42339,409940971687014400,1386572518,SaintGutFree,Я лишь спрашиваю себя почему я еще не послала ...,1,0,0,0,14156,76,31,3
218556,423922666123579392,1389906014,dimasashurin,Я прям афиши вижу типо: СТИХИ КАБАНА. ЭКСКЛЮЗИ...,-1,0,0,0,4079,13,47,0


Проведем минимальную предобработку: уберем упоминания других пользователей, начинающихся с @ и соответствующую метку ретвита RT (то, что их очень много, можно увидеть в нескольких выведенных строках сверху). Также в текстах интернета часто могут быть какие-либо ссылки и е-мейлы, их также уберем (регулярки для емейлов и ссылок были найдены в интернете)

In [3]:
def small_preprocessing(x):
  x = re.sub(r'@[a-zA-Z_0-9]+', '', x)
  x = re.sub(r'RT', '', x)
  url_pattern = re.compile(r'((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*')
  email_pattern = re.compile(r'[\w.-]+@[\w.-]+.\w+')
  for pattern in (url_pattern, email_pattern):
    x = pattern.sub('', x)
  return x

Колонку с целевой переменной type преобразуем из значений -1 и 1 в привычные 0 и 1

In [4]:
df['type']= df['type']>0
df['type'] = df['type'].astype(int)

In [5]:
df['type'].value_counts()

1    114911
0    111923
Name: type, dtype: int64

Положительный и отрицательный классы сбалансированы

In [6]:
texts = df['text'].apply(small_preprocessing)
target = df['type'].values

Делим данные на тренировочную и тестовую выборки (тестовая будет выполнять роль валидационной, на ней будем смотреть качество)

In [7]:
train_data, test_data, train_labels, test_labels = train_test_split(texts, target, test_size=0.33, random_state=42, stratify=target)

## 1. biLSTM

### Обучаемый Embedding слой

Преобразование токенов в идентификаторы можно делать не вручную, а через Tokinizer из keras. Он точно так же строит на корпусе текстов словарь word2id и маппит индексы к словами. Паддинги и oov слова также учитываются: для паддинга резервируется индекс 0, для oov - 1. Надо лишь отдельно указать, на какую последовательность символов заменять oov слова (на unk)


Tokenizer также игнорирует знаки препинания и переводит текст в нижний регистр. Поэтому отдельную такую предобработку выше мы не делали. Лемматизацию проводить пока не будем, поскольку эмбеддинги обучаются самостоятельно, и, в принципе,  из-за лемматизации можно также потерять некоторые слова (какие-то особенные слова из твиттера могут не так лемматизироваться) 

Тokenizer фитится только на тренировочной выборке, поскольку в реальной жизни модель никогда заранее не узнает, какие будут тестовые данные, и как их надо специально подготовить.

In [8]:
tokenizer = Tokenizer(oov_token='unk')
tokenizer.fit_on_texts(train_data)

Посмотрим сколько уникальных токенов имеется в нашем распоряжении

In [9]:
len(tokenizer.word_counts)

147627

In [10]:
#выберем не очень большой размер эмбеддинга, который будет обучаться, чтобы не ждать долго обучения и не занимать много памяти
embedding_size = 20
word2index = tokenizer.word_index

In [11]:
tokenized_train = tokenizer.texts_to_sequences(train_data)
tokenized_test = tokenizer.texts_to_sequences(test_data)

In [12]:
tokenized_train[0]

[24610,
 60174,
 24611,
 60175,
 3660,
 60176,
 24612,
 2438,
 39794,
 1595,
 73,
 651,
 18086,
 2936,
 1109]

Добавим паддинги так, чтобы все последовательности имели длину, равную максимальной

In [13]:
maxlen = max([len(x) for x in tokenized_train])
print('Максимальная длина предложения: ', maxlen)

padded_train = pad_sequences(tokenized_train, maxlen=maxlen)
padded_test = pad_sequences(tokenized_test, maxlen=maxlen)

Максимальная длина предложения:  41


In [14]:
padded_train[0]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0, 24610,
       60174, 24611, 60175,  3660, 60176, 24612,  2438, 39794,  1595,
          73,   651, 18086,  2936,  1109], dtype=int32)

Собираем модель из Embedding слоя, biLSTM и полносвязного.

(задание можно прочитать двусмысленно: после biLSTM один полносвязный слой, который является же выходным, либо два полносвязных, из которых один имеет произвольное число узлов, а другой - выходной. Я остановилась на первом варианте, потому что так было во всех семинарах и никакой разницы во времени/качестве между этими разными реализациями фактически нет, по крайней мере мне не удалось их обнаружить)

In [15]:
input_layer = Input((maxlen,))
embedding_layer = Embedding(input_dim=len(word2index), output_dim=embedding_size, mask_zero=True)(input_layer)
lstm = Bidirectional(LSTM(128, return_sequences=False))(embedding_layer)
output_layer = Dense(1, activation="sigmoid")(lstm)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss='binary_crossentropy',
                  optimizer=Adam(learning_rate=0.001),
                  metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
print(model.summary())

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 41)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 41, 20)            2952560   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               152576    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 3,105,393
Trainable params: 3,105,393
Non-trainable params: 0
_________________________________________________________________
None


Не будем ставить большое количество эпох. Трех будет достаточно, чтобы посмотреть на то, как уменьшается ли ошибка и есть ли склонность к переобучению. Батч сайз подобрала на основе наблюдений и решенных ранее задач

In [ ]:
model.fit(padded_train, train_labels, 
          validation_data=(padded_test, test_labels), 
          batch_size=128, epochs = 3, verbose = 1)


Train on 151978 samples, validate on 74856 samples
Epoch 1/3
151978/151978 [==============================] - 134s 880us/step - loss: 0.5059 - accuracy: 0.7414 - precision: 0.6992 - recall: 0.7444 - val_loss: 0.4760 - val_accuracy: 0.7625 - val_precision: 0.7568 - val_recall: 0.7324
Epoch 2/3
151978/151978 [==============================] - 135s 886us/step - loss: 0.4026 - accuracy: 0.8122 - precision: 0.7807 - recall: 0.7443 - val_loss: 0.4822 - val_accuracy: 0.7678 - val_precision: 0.7896 - val_recall: 0.7535
Epoch 3/3
151978/151978 [==============================] - 133s 878us/step - loss: 0.3428 - accuracy: 0.8445 - precision: 0.8000 - recall: 0.7635 - val_loss: 0.5032 - val_accuracy: 0.7607 - val_precision: 0.8038 - val_recall: 0.7722


Лосс уменьшается достаточно быстро, показатели качества достаточно высокие и на обучающей и на валидационной выборке. НО: можно заметить, что значение лосса на валидационной выборке с каждой эпохой все увеличивается, а accuracy уменьшается. Это верный признак переобучения, что является огромным минусом это реализации

### Предобученные FastText эмбеддинги

In [30]:
!wget https://rusvectores.org/static/models/rusvectores4/fasttext/araneum_none_fasttextcbow_300_5_2018.tgz

--2021-02-21 18:37:34--  https://rusvectores.org/static/models/rusvectores4/fasttext/araneum_none_fasttextcbow_300_5_2018.tgz
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2691248108 (2.5G) [application/x-gzip]
Saving to: ‘araneum_none_fasttextcbow_300_5_2018.tgz’

araneum_none_fastte 100%[===================>]   2.51G  31.4MB/s    in 82s     

2021-02-21 18:38:56 (31.5 MB/s) - ‘araneum_none_fasttextcbow_300_5_2018.tgz’ saved [2691248108/2691248108]



In [31]:
!tar -xf araneum_none_fasttextcbow_300_5_2018.tgz

In [32]:
ft_file = 'araneum_none_fasttextcbow_300_5_2018.model'
ft_vectors = KeyedVectors.load(ft_file)

Загрузили предобученную модель fasttext для русского языка. Для того, чтобы большему количеству слов сопоставить вектор из модели, тексты необходимо лемматизировать. Иначе только те слова, что по счастливому стечению обстоятельств имеют начальную форму в текстах, будут иметь качественное векторное представление.

In [33]:
%%time
#ячейка работает довольно долго
punkt = punctuation + '«»—…“”*№–'
morph = pymorphy2.MorphAnalyzer()

def lemmatizing(text):
  clean_text = [word.strip(punkt) for word in text.lower().split()]
  lem_text = ' '.join([morph.parse(word)[0].normal_form for word in clean_text])
  return lem_text


lem_train_data = train_data.apply(lemmatizing)
lem_test_data = test_data.apply(lemmatizing)

CPU times: user 7min 44s, sys: 104 ms, total: 7min 45s
Wall time: 7min 47s


Переводим тексты в последовательность идентификатором с помощью Tokenizer-а на лемматизированных данных. Добавляем паддинги. 

In [34]:
lem_tokenizer = Tokenizer(oov_token='unk')
lem_tokenizer.fit_on_texts(lem_train_data)
len(lem_tokenizer.word_counts)

98752

Уникальных слов стало меньше благодаря лемматизации

In [35]:
lem_tokenized_train = lem_tokenizer.texts_to_sequences(lem_train_data)
lem_tokenized_test = lem_tokenizer.texts_to_sequences(lem_test_data)
lem_padded_train = pad_sequences(lem_tokenized_train, maxlen=maxlen)
lem_padded_test = pad_sequences(lem_tokenized_test, maxlen=maxlen)

In [36]:
#достаем из объекта Tokenizer словарь word2id
lemma2index = lem_tokenizer.word_index
#фиксируем размер эмбеддингов
ft_embs_size = ft_vectors.vector_size
#создаем матрицу размера (кол-во слов в корпусе; размер эмбеддинга). заполняем ее случайным образом из нормального распределения 
#(для этого необходимо посчитать среднее и корень из дисперсии всех векторов в fasttext модели)
emb_mean, emb_std = ft_vectors.syn0.mean(), ft_vectors.syn0.std()
embedding_matrix = np.random.normal(emb_mean, emb_std, (len(lemma2index)+1, ft_embs_size)) #единичка прибавляется, потому что индекса 0 в словаре word2index нет
for word, i in lemma2index.items():
    if word in ft_vectors:
      embedding_vector = ft_vectors[word]
      embedding_matrix[i] = embedding_vector

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  import sys


In [37]:
embedding_matrix.shape

(98754, 300)

Первая размерность на 2 больше, чем количество уникальных слов, указанных выше, из-за паддинга и UNK.

Строим саму сеть (изменились лишь параметры в embedding слое)

In [ ]:
input_layer = Input((maxlen,))
embedding_layer = Embedding(input_dim=len(lemma2index)+1, output_dim=ft_embs_size, 
                            mask_zero=True, weights=[embedding_matrix],
                            trainable=False)(input_layer)
lstm = Bidirectional(LSTM(128, return_sequences=False))(embedding_layer)
output_layer = Dense(1, activation="sigmoid")(lstm)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss='binary_crossentropy',
                  optimizer=Adam(learning_rate=0.001),
                  metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
print(model.summary())

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 41)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 41, 300)           29626200  
_________________________________________________________________
bidirectional_2 (Bidirection (None, 256)               439296    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 30,065,753
Trainable params: 439,553
Non-trainable params: 29,626,200
_________________________________________________________________
None


In [ ]:
model.fit(lem_padded_train, train_labels, 
          validation_data=(lem_padded_test, test_labels), 
          batch_size=128, epochs = 3, verbose = 1)

Train on 151978 samples, validate on 74856 samples
Epoch 1/3
151978/151978 [==============================] - 130s 857us/step - loss: 0.5539 - accuracy: 0.7041 - precision_1: 0.7058 - recall_1: 0.6505 - val_loss: 0.5361 - val_accuracy: 0.7184 - val_precision_1: 0.7196 - val_recall_1: 0.6898
Epoch 2/3
151978/151978 [==============================] - 127s 837us/step - loss: 0.5208 - accuracy: 0.7283 - precision_1: 0.7235 - recall_1: 0.7005 - val_loss: 0.5311 - val_accuracy: 0.7215 - val_precision_1: 0.7248 - val_recall_1: 0.7117
Epoch 3/3
151978/151978 [==============================] - 124s 816us/step - loss: 0.5044 - accuracy: 0.7403 - precision_1: 0.7258 - recall_1: 0.7196 - val_loss: 0.5095 - val_accuracy: 0.7383 - val_precision_1: 0.7309 - val_recall_1: 0.7205


Loss убывает гораздо медленнее, чем в предыдущей сетке, качество, соответственно, тоже ниже. Однако надо обратить внимание на то, что у этой сети почти в три раза больше обучающихся параметров у слоя biLSTM (который по сути основной) - 439 тысяч vs 152 тысячи. Поэтому 3 эпохи - это достаточно мало, и не дает никаких оснований, чтобы говорить о том, что вторая сеть показывает худшие результаты, чем первая. Попробуем еще немного ее поучить: 

In [ ]:
model.fit(lem_padded_train, train_labels, 
          validation_data=(lem_padded_test, test_labels), 
          batch_size=128, epochs = 4, verbose = 1)

Train on 151978 samples, validate on 74856 samples
Epoch 1/4
151978/151978 [==============================] - 127s 836us/step - loss: 0.4904 - accuracy: 0.7505 - precision_1: 0.7350 - recall_1: 0.7220 - val_loss: 0.5070 - val_accuracy: 0.7415 - val_precision_1: 0.7371 - val_recall_1: 0.7258
Epoch 2/4
151978/151978 [==============================] - 123s 812us/step - loss: 0.4781 - accuracy: 0.7596 - precision_1: 0.7393 - recall_1: 0.7297 - val_loss: 0.5026 - val_accuracy: 0.7419 - val_precision_1: 0.7422 - val_recall_1: 0.7303
Epoch 3/4
151978/151978 [==============================] - 125s 823us/step - loss: 0.4649 - accuracy: 0.7693 - precision_1: 0.7452 - recall_1: 0.7315 - val_loss: 0.5001 - val_accuracy: 0.7481 - val_precision_1: 0.7477 - val_recall_1: 0.7329
Epoch 4/4
151978/151978 [==============================] - 124s 814us/step - loss: 0.4508 - accuracy: 0.7789 - precision_1: 0.7503 - recall_1: 0.7347 - val_loss: 0.5032 - val_accuracy: 0.7495 - val_precision_1: 0.7522 - val_re

Качество с каждой эпохой повышается и на обучающей и на валидационной выборке, лосс также равномерно убывает. Это говорит о том, что сеть продолжает обучаться. Попробуем пройти еще через 4 эпохи:

In [ ]:
model.fit(lem_padded_train, train_labels, 
          validation_data=(lem_padded_test, test_labels), 
          batch_size=128, epochs = 4, verbose = 1)

Train on 151978 samples, validate on 74856 samples
Epoch 1/4
151978/151978 [==============================] - 126s 831us/step - loss: 0.4355 - accuracy: 0.7886 - precision_1: 0.7543 - recall_1: 0.7392 - val_loss: 0.5053 - val_accuracy: 0.7499 - val_precision_1: 0.7562 - val_recall_1: 0.7412
Epoch 2/4
151978/151978 [==============================] - 126s 829us/step - loss: 0.4180 - accuracy: 0.7991 - precision_1: 0.7583 - recall_1: 0.7432 - val_loss: 0.5227 - val_accuracy: 0.7466 - val_precision_1: 0.7605 - val_recall_1: 0.7442
Epoch 3/4
151978/151978 [==============================] - 125s 823us/step - loss: 0.3985 - accuracy: 0.8100 - precision_1: 0.7630 - recall_1: 0.7455 - val_loss: 0.5263 - val_accuracy: 0.7436 - val_precision_1: 0.7645 - val_recall_1: 0.7477
Epoch 4/4
151978/151978 [==============================] - 126s 828us/step - loss: 0.3776 - accuracy: 0.8237 - precision_1: 0.7663 - recall_1: 0.7502 - val_loss: 0.5503 - val_accuracy: 0.7436 - val_precision_1: 0.7683 - val_re

На этом, пожалуй, стоит остановиться, потому что качество на валидационной выборке перестало расти, и валидационный лосс только повышается - признаки переобучения. Показатели качества достигли хорошей отметки: на тренировочной выборке accuracy почти догнал результат предыдущей сети (82 и 84). Точность и полнота немного ниже (точность на 4 процента, полнота на 1). Все значения метрик на валидационной выборке находятся в районе 75 процентов, что тоже на 2-3 процента ниже результата, полученной предыдущей сетью. Такая разница все же довольно большая, кроме того дополнительно тратится время и ресурсы на лемматизацию и загрузку модели.

Но огромным преимуществом является отсутствие такого сильного переобучения, как в первом случае, где лосс на валидационной выборке сразу же начал расти. Здесь он плавно убывает, и только на 7-8 эпохах начинает стоять на одном месте, а затем расти

### Обучаемые FastText эмбеддинги

Обучим с нуля Fasttext эмбеддинги на лемматизированной тренировочной выборке. Тренировочной - исходя из соображений, что мы не должны, по идее, заранее видеть тестовые данные. Лемматизированной - потому что уникальных слов в ней меньше, удалены знаки препинания, соответственно, модель будет немного легче. Если обучать на неподготовленных данных, то велик шанс, что в колабе кончится ОЗУ

Размер эмбеддинга тоже подбирался исходя из того, на сколько хватит памяти колаба (200 уже вылетает)

In [ ]:
new_ft = FastText(lem_train_data, size=150, iter=5)

Точно так же, как и с предобученными эмбеддингами, создаем матрицу размера (размер словаря; размер вектора)

In [ ]:
emb_mean, emb_std = new_ft.wv.syn0.mean(), new_ft.wv.syn0.std()

ft_embs_size = new_ft.vector_size
new_embedding_matrix = np.random.normal(emb_mean, emb_std, (len(lemma2index)+1, ft_embs_size))
for word, i in lemma2index.items():
    if word in new_ft:
      embedding_vector = new_ft[word]
      new_embedding_matrix[i] = embedding_vector

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


Строим точно такую же сетку:

In [ ]:
input_layer = Input((maxlen,))
embedding_layer = Embedding(input_dim=len(lemma2index)+1, output_dim=ft_embs_size, 
                            mask_zero=True, weights=[new_embedding_matrix],
                            trainable=False)(input_layer)
lstm = Bidirectional(LSTM(128, return_sequences=False))(embedding_layer)
output_layer = Dense(1, activation="sigmoid")(lstm)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss='binary_crossentropy',
                  optimizer=Adam(learning_rate=0.001),
                  metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
print(model.summary())

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 41)                0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 41, 150)           14813100  
_________________________________________________________________
bidirectional_3 (Bidirection (None, 256)               285696    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 15,099,053
Trainable params: 285,953
Non-trainable params: 14,813,100
_________________________________________________________________
None


In [ ]:
model.fit(lem_padded_train, train_labels, 
          validation_data=(lem_padded_test, test_labels), 
          batch_size=128, epochs = 3, verbose = 1)


Train on 151978 samples, validate on 74856 samples
Epoch 1/3
151978/151978 [==============================] - 124s 818us/step - loss: 0.6310 - accuracy: 0.5904 - precision_2: 0.6022 - recall_2: 0.5368 - val_loss: 0.6298 - val_accuracy: 0.5978 - val_precision_2: 0.6156 - val_recall_2: 0.5168
Epoch 2/3
151978/151978 [==============================] - 122s 805us/step - loss: 0.6273 - accuracy: 0.5976 - precision_2: 0.6191 - recall_2: 0.5152 - val_loss: 0.6278 - val_accuracy: 0.6004 - val_precision_2: 0.6206 - val_recall_2: 0.5172
Epoch 3/3
151978/151978 [==============================] - 122s 805us/step - loss: 0.6254 - accuracy: 0.6015 - precision_2: 0.6222 - recall_2: 0.5171 - val_loss: 0.6270 - val_accuracy: 0.6003 - val_precision_2: 0.6242 - val_recall_2: 0.5153


3 эпохи опять же достаточно мало для этой сети. Но можно заметить, что лосс уменьшается очень медленно, и качество пока что достаточно низкое, по сравнению с предыдущими двумя реализациями, особенно низкая полнота. Попробуем пообучать еще 5 эпох

In [ ]:
model.fit(lem_padded_train, train_labels, 
          validation_data=(lem_padded_test, test_labels), 
          batch_size=128, epochs = 5, verbose = 1)

Train on 151978 samples, validate on 74856 samples
Epoch 1/5
151978/151978 [==============================] - 122s 801us/step - loss: 0.6241 - accuracy: 0.6025 - precision_2: 0.6257 - recall_2: 0.5129 - val_loss: 0.6265 - val_accuracy: 0.6011 - val_precision_2: 0.6270 - val_recall_2: 0.5122
Epoch 2/5
151978/151978 [==============================] - 122s 800us/step - loss: 0.6231 - accuracy: 0.6037 - precision_2: 0.6279 - recall_2: 0.5118 - val_loss: 0.6254 - val_accuracy: 0.6045 - val_precision_2: 0.6285 - val_recall_2: 0.5124
Epoch 3/5
151978/151978 [==============================] - 122s 801us/step - loss: 0.6218 - accuracy: 0.6060 - precision_2: 0.6289 - recall_2: 0.5135 - val_loss: 0.6281 - val_accuracy: 0.5952 - val_precision_2: 0.6302 - val_recall_2: 0.5106
Epoch 4/5
151978/151978 [==============================] - 121s 794us/step - loss: 0.6205 - accuracy: 0.6089 - precision_2: 0.6315 - recall_2: 0.5080 - val_loss: 0.6260 - val_accuracy: 0.6039 - val_precision_2: 0.6323 - val_re

Тенденции не поменялись: лосс на обеих выборках убывает очень медленно, качество низкое, хотя и повышается на несколько сотых после каждой эпохи. Это говорит о том, что нашими весами не получается достаточно хорошо минимизировать функцию ошибки.

Попробуем обучить эмбеддинги Fasttext с нуля на нелемматизированных данных: вдруг результат будет лучше. Тогда все равно нужно избавиться от пунктуации и верхнего регистра:

In [ ]:
punkt = punctuation + '«»—…“”*№–'

def preprocess(text):
  clean_text = ' '.join([word.strip(punkt) for word in text.lower().split()])
  return clean_text

clean_train_data = train_data.apply(preprocess)
clean_test_data = test_data.apply(preprocess)

Обучаем эмбеддинги с теми же параметрами и создаем матрицу весов

In [ ]:
new_ft = FastText(clean_train_data, size=150, iter=5)

In [ ]:
emb_mean, emb_std = new_ft.wv.syn0.mean(), new_ft.wv.syn0.std()

ft_embs_size = new_ft.vector_size
new_embedding_matrix = np.random.normal(emb_mean, emb_std, (len(word2index)+1, ft_embs_size))
for word, i in word2index.items():
    if word in new_ft:
      embedding_vector = new_ft[word]
      new_embedding_matrix[i] = embedding_vector

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


Формируем сеть и обучаем ее так же 8 эпох

In [ ]:
input_layer = Input((maxlen,))
embedding_layer = Embedding(input_dim=len(word2index)+1, output_dim=ft_embs_size, 
                            mask_zero=True, weights=[new_embedding_matrix],
                            trainable=False)(input_layer)
lstm = Bidirectional(LSTM(128, return_sequences=False))(embedding_layer)
output_layer = Dense(1, activation="sigmoid")(lstm)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss='binary_crossentropy',
                  optimizer=Adam(learning_rate=0.001),
                  metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])


model.fit(padded_train, train_labels, 
          validation_data=(padded_test, test_labels), 
          batch_size=128, epochs = 8, verbose = 1)

Train on 151978 samples, validate on 74856 samples
Epoch 1/8
151978/151978 [==============================] - 122s 803us/step - loss: 0.6336 - accuracy: 0.5862 - precision_1: 0.6168 - recall_1: 0.4562 - val_loss: 0.6325 - val_accuracy: 0.5891 - val_precision_1: 0.6236 - val_recall_1: 0.4638
Epoch 2/8
151978/151978 [==============================] - 121s 797us/step - loss: 0.6296 - accuracy: 0.5949 - precision_1: 0.6222 - recall_1: 0.4803 - val_loss: 0.6297 - val_accuracy: 0.5946 - val_precision_1: 0.6265 - val_recall_1: 0.4752
Epoch 3/8
151978/151978 [==============================] - 120s 790us/step - loss: 0.6272 - accuracy: 0.5975 - precision_1: 0.6290 - recall_1: 0.4736 - val_loss: 0.6283 - val_accuracy: 0.5984 - val_precision_1: 0.6300 - val_recall_1: 0.4762
Epoch 4/8
151978/151978 [==============================] - 120s 788us/step - loss: 0.6254 - accuracy: 0.6010 - precision_1: 0.6307 - recall_1: 0.4789 - val_loss: 0.6272 - val_accuracy: 0.5996 - val_precision_1: 0.6337 - val_re

Различия действительно есть: значение лосса опустилось немного ниже, чем в варианте с лемматизированными данными. Немного выше значения accuracy и точности и на тренировочной и на тестовой выборках. Зато полнота очень низкая. Классификатор предсказывает верно малое количество предложений, относящихся к положительному классу.

Разница в результатах, полученными ранее, составляет более 10 процентов по всем метрикам - это очень большое значение.

### Снова обучаемый Embedding слой

Высокие показатели метрик были получены с помощью обучения эмбеддингов внутри модели. Этот вариант оказался также самым быстрым (предобученные эмбеддинги не сильно отстали по качеству, но зато значительно по подготовке и по времени обучения). 


Для справедливости нужно попробовать реализовать этот вариант еще и  на лемматизированных данных, вдруг он покажет себя очень плохо или, наоборот, еще лучше. Также увеличим размер эмбеддинга, чтобы можно было сопоставить с fasttext. 
Добавим в архитектуру дропаут, чтобы минимизировать переобучение, поскольку модель становится уже тяжелой

In [ ]:
input_layer = Input((maxlen,))
embedding_layer = Embedding(input_dim=len(lemma2index)+1, output_dim=150, mask_zero=True)(input_layer)
lstm = Bidirectional(LSTM(128, return_sequences=False))(embedding_layer)
d = Dropout(0.2)(lstm)
output_layer = Dense(1, activation="sigmoid")(d)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss='binary_crossentropy',
                  optimizer=Adam(learning_rate=0.001),
                  metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
print(model.summary())

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 41)                0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 41, 150)           14813100  
_________________________________________________________________
bidirectional_4 (Bidirection (None, 256)               285696    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 15,099,053
Trainable params: 15,099,053
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.fit(lem_padded_train, train_labels, 
          validation_data=(lem_padded_test, test_labels), 
          batch_size=128, epochs = 3, verbose = 1)

Train on 151978 samples, validate on 74856 samples
Epoch 1/3
151978/151978 [==============================] - 134s 881us/step - loss: 0.4976 - accuracy: 0.7478 - precision_3: 0.7176 - recall_3: 0.7418 - val_loss: 0.4648 - val_accuracy: 0.7721 - val_precision_3: 0.7607 - val_recall_3: 0.7456
Epoch 2/3
151978/151978 [==============================] - 133s 875us/step - loss: 0.3552 - accuracy: 0.8404 - precision_3: 0.7867 - recall_3: 0.7663 - val_loss: 0.4882 - val_accuracy: 0.7663 - val_precision_3: 0.7974 - val_recall_3: 0.7785
Epoch 3/3
151978/151978 [==============================] - 133s 872us/step - loss: 0.2342 - accuracy: 0.9003 - precision_3: 0.8111 - recall_3: 0.7940 - val_loss: 0.5698 - val_accuracy: 0.7576 - val_precision_3: 0.8193 - val_recall_3: 0.8049


Всего за три эпохи лосс достиг маленького значения, аccuracy увеличилось до 90 на, точность и полнота примерно до 80 на тренировочной выборке, а на тестовой наблюдаем высокие значения точности и полноты (эти результаты выше, чем на сетке с нелемматизированными данными), но зато не очень большое accuracy (примерно такое же, как и в первом варианте)

Loss на валидационной выборке так же растет, как и с меньшей размерностью эмбеддинга. Это большой минус у этой реализации. Мне не удалось с помощью добавления дропаута и регуляризации заставить ошибку на валидационной выборке убывать, поэтому во второй части задания я буду использовать предобученные эмбеддинги. Может быть, будет проигрыш в несколько процентов в качестве и во времени подготовки данных, но зато модель будет более стабильна, а это очень важно

## 2. biLSTM + charCNN

Создадим отфильтрованный список лемм: в него войдут только те, что встречались в корпусе более пяти раз. Таким образом мы отсеем редкие, незначительные символы и какой-то лишний мусор

In [38]:
filtered_vocab = []

for k,v in lem_tokenizer.word_counts.items():
  if v > 5:
    filtered_vocab.append(k)

In [39]:
chars = set([letter for word in filtered_vocab for letter in word])
print(chars)

{'ғ', 'т', 'ф', 'e', 'щ', '0', 'a', 'в', 'э', 'h', 'х', '1', '•', 'б', 'n', '3', 'ў', 'u', 'з', 'ь', 'f', 'ъ', 'й', 'о', '❤', '❄', '4', 'q', 'b', '2', 'ї', 'ч', '6', 'd', 'ж', 'и', '☆', '7', 'с', 'v', 'p', 'қ', 'ы', '©', 'j', 'у', 'ө', '♥', 'є', 'е', '°', 'к', 'п', '✌', 'ц', 'ё', 'і', '8', 'l', 'я', 'д', 'н', '9', '✔', 'i', 'r', 'ү', '️', 'ë', 'y', 'w', 'm', '☺', 'м', 'c', 'x', 's', 'k', 'g', '5', 'а', 'ә', 'ю', 'ң', 'л', '♡', 't', 'o', 'ш', 'z', 'г', 'р'}


Помимо русских и английских букв присутствуют эмодзи, цифры и буквы из каких-то других алфавитов. Встречаемость даже странных символов очень высокая. 

Создадим словарь char2id и определим максимальную длину слова, чтобы дополнять паддингами все слова до этого значения

In [40]:
char2id = {c: i + 2 for i, c in enumerate(chars)}
char2id["pad"] = 0
char2id["unk"] = 1

char_maxlen = max(len(x) for x in filtered_vocab)
print("Максимальная длина слова:", char_maxlen)

Максимальная длина слова: 33


Функция из семинара, которая трансформирует корпус текстов в трехмерный массив, где каждый обучающий пример представлен набором идентификаторов токенов, а каждый токен набором идентификаторов символов:

In [41]:
def make_X_char(sentences):
  X_char = []
  for sentence in sentences:
      sent_seq = []
      for i in range(maxlen):
          word_seq = []
          for j in range(char_maxlen):
              try:
                  word_seq.append(char2id[sentence[i][j].lower()])
              except:
                  word_seq.append(char2id["pad"])
          sent_seq.append(word_seq)
      X_char.append(np.array(sent_seq))
  return np.array(X_char)

Преобразуем в этот формат лемматизированные обучающую и тестовую выборки

In [42]:
char_train, char_test = make_X_char(lem_train_data), make_X_char(lem_test_data)

In [43]:
char_train.shape

(151978, 41, 33)

Строим саму сеть с двумя входами, сверточным слоем и двумя двусторонними LSTM: 

In [44]:
word_in = Input((maxlen,))
emb_word = Embedding(input_dim=len(lemma2index)+1, output_dim=ft_embs_size, 
                     mask_zero=True, weights=[embedding_matrix], trainable=False)(word_in)

f_lstm = Bidirectional(LSTM(units=64, return_sequences=False, recurrent_dropout=0.3))(emb_word)

char_in = Input((maxlen, char_maxlen,))
emb_char = TimeDistributed(Embedding(input_dim=len(char2id), output_dim=10, input_length=char_maxlen))(char_in)
char_enc = TimeDistributed(Conv1D(filters=12, kernel_size=3))(emb_char)
char_flat = TimeDistributed(Flatten())(char_enc)
s_lstm = Bidirectional(LSTM(units=64, return_sequences=False, recurrent_dropout=0.3))(char_flat)

x = concatenate([f_lstm, s_lstm])
out = Dense(1, activation="sigmoid")(x)

model = Model(inputs=[char_in, word_in], outputs=out)


model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])


model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, 41, 33)       0                                            
__________________________________________________________________________________________________
time_distributed_16 (TimeDistri (None, 41, 33, 10)   940         input_12[0][0]                   
__________________________________________________________________________________________________
input_11 (InputLayer)           (None, 41)           0                                            
__________________________________________________________________________________________________
time_distributed_17 (TimeDistri (None, 41, 31, 12)   372         time_distributed_16[0][0]        
____________________________________________________________________________________________

In [45]:
model.fit([char_train, lem_padded_train], train_labels, 
          validation_data=([char_test, lem_padded_test], test_labels), 
          batch_size=128, epochs = 5, verbose = 1)

Train on 151978 samples, validate on 74856 samples
Epoch 1/5
151978/151978 [==============================] - 221s 1ms/step - loss: 0.5596 - accuracy: 0.6998 - precision_4: 0.6865 - recall_4: 0.6576 - val_loss: 0.5394 - val_accuracy: 0.7193 - val_precision_4: 0.7183 - val_recall_4: 0.6816
Epoch 2/5
151978/151978 [==============================] - 216s 1ms/step - loss: 0.5276 - accuracy: 0.7233 - precision_4: 0.7243 - recall_4: 0.6900 - val_loss: 0.5270 - val_accuracy: 0.7257 - val_precision_4: 0.7264 - val_recall_4: 0.6989
Epoch 3/5
151978/151978 [==============================] - 215s 1ms/step - loss: 0.5145 - accuracy: 0.7320 - precision_4: 0.7280 - recall_4: 0.7058 - val_loss: 0.5121 - val_accuracy: 0.7364 - val_precision_4: 0.7328 - val_recall_4: 0.7050
Epoch 4/5
151978/151978 [==============================] - 213s 1ms/step - loss: 0.5053 - accuracy: 0.7398 - precision_4: 0.7367 - recall_4: 0.7052 - val_loss: 0.5078 - val_accuracy: 0.7398 - val_precision_4: 0.7380 - val_recall_4: 

Лосс на обеих выборках уменьшается, причем не так медленно, как это было с fasttext эмбеддингами на biLSTM. Значения метрик умеренно высокие: accuracy и точность около 74 на обеих выборках, и полнота 71. Если в точности и полноте мы роиграли модели с обучаемыми в процессе эмбеддингами, то accuracy на валидационной выборке у нее был примерно тот же. Зато, как я писала выше, в этом варианте реализации отсутствует такое сильное переобучение (есть только признаки его начала на 4 эпохе).

Усложним архитектуру, чтобы попробовать получить более высокое качество. Я удвоила количество юнитов в обоих LSTM слоях, количество фильтров на сверточном слое, после него добавила дропаут.
После слоя конкатенации добавила полносвязный слой. Основная мотивация в том, чтобы усложить сетку , чтобы она смогла выявить более сложные зависимости в данных, но и избежать переобучения с помощью дропаута. 

In [52]:
word_in = Input((maxlen,))
emb_word = Embedding(input_dim=len(lemma2index)+1, output_dim=ft_embs_size, 
                     mask_zero=True, weights=[embedding_matrix], trainable=False)(word_in)
f_lstm = Bidirectional(LSTM(units=128, return_sequences=False, recurrent_dropout=0.3))(emb_word)

char_in = Input((maxlen, char_maxlen,))
emb_char = TimeDistributed(Embedding(input_dim=len(char2id), output_dim=10, input_length=char_maxlen))(char_in)
char_enc = TimeDistributed(Conv1D(filters=24, kernel_size=3))(emb_char)
char_enc = Dropout(0.5)(char_enc)
char_flat = TimeDistributed(Flatten())(char_enc)
s_lstm = Bidirectional(LSTM(units=128, return_sequences=False, recurrent_dropout=0.3))(char_flat)

x = concatenate([f_lstm, s_lstm])
x = Dense(64, activation='relu')(x)
out = Dense(1, activation="sigmoid")(x)

model = Model(inputs=[char_in, word_in], outputs=out)


model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])


model.summary()

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_20 (InputLayer)           (None, 41, 33)       0                                            
__________________________________________________________________________________________________
time_distributed_28 (TimeDistri (None, 41, 33, 10)   940         input_20[0][0]                   
__________________________________________________________________________________________________
time_distributed_29 (TimeDistri (None, 41, 31, 24)   744         time_distributed_28[0][0]        
__________________________________________________________________________________________________
input_19 (InputLayer)           (None, 41)           0                                            
____________________________________________________________________________________________

In [53]:
model.fit([char_train, lem_padded_train], train_labels, 
          validation_data=([char_test, lem_padded_test], test_labels), 
          batch_size=128, epochs = 5, verbose = 1)

Train on 151978 samples, validate on 74856 samples
Epoch 1/5
151978/151978 [==============================] - 227s 1ms/step - loss: 0.5512 - accuracy: 0.7047 - precision_7: 0.6985 - recall_7: 0.6500 - val_loss: 0.5299 - val_accuracy: 0.7231 - val_precision_7: 0.7213 - val_recall_7: 0.6903
Epoch 2/5
151978/151978 [==============================] - 222s 1ms/step - loss: 0.5221 - accuracy: 0.7268 - precision_7: 0.7255 - recall_7: 0.7011 - val_loss: 0.5223 - val_accuracy: 0.7287 - val_precision_7: 0.7270 - val_recall_7: 0.7103
Epoch 3/5
151978/151978 [==============================] - 220s 1ms/step - loss: 0.5095 - accuracy: 0.7356 - precision_7: 0.7287 - recall_7: 0.7167 - val_loss: 0.5215 - val_accuracy: 0.7286 - val_precision_7: 0.7340 - val_recall_7: 0.7125
Epoch 4/5
151978/151978 [==============================] - 219s 1ms/step - loss: 0.4966 - accuracy: 0.7456 - precision_7: 0.7385 - recall_7: 0.7103 - val_loss: 0.5014 - val_accuracy: 0.7436 - val_precision_7: 0.7409 - val_recall_7: 

Сильного улучшения не наблюдается, однако все же метрики на валидационной выборке немного повысились: accuracy на 1 процент, точность на и полнота на чуть меньше. Возможно, если подождать еще несколько эпох, то значения будут еще выше. 